In [1]:
from pathlib import Path
from typing import Callable, Dict, Optional
import numpy as np
import sys
sys.path.append('..')
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from skimage import io
from torchvision.transforms import ToTensor, Resize, CenterCrop
from data_handling.base import BaseDataModuleClass
from datetime import datetime
from data_handling.caching import SharedCache
from hydra import compose, initialize
# from data_handling.chest_xray import MimicDataset, MimicDataModule
from data_handling.sampler import SamplerFactory
from torch.utils.data import DataLoader
from causal_models.train_setup import setup_dataloaders
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
class Hparams:
    def update(self, dict):
        for k, v in dict.items():
            setattr(self, k, v)

args = Hparams()
args.hps = "rsna224_224_with_seg"
args.parents_x = ["finding", "sex", "age"]
args.batch_size = 1
dataloaders = setup_dataloaders(args=args, cache=False, shuffle_train=False)

{'project_name': 'chest X ray generation', 'seed': 11, 'mixed_precision': False, 'is_unit_test_config': False, 'data': {'batch_size': 1, 'num_workers': 12, 'pin_memory': True, 'input_channels': 1, 'weights': 'None', 'augmentations': {'resize': [224, 224], 'center_crop': 'None', 'random_rotation': 0, 'horizontal_flip': False, 'vertical_flip': False, 'random_crop': 'None', 'random_color_jitter': 0.1, 'random_erase_scale': [0.0, 0.0], 'sharp': 0.0}, 'prop_train': 1.0, '_target_': 'data_handling.rsna.RSNAPneumoniaDataModule', 'dataset': 'rsna', 'domain': 'None', 'cache': False, 'seg_target_list': ['Left-Lung', 'Right-Lung', 'Heart']}, 'trainer': {'name': 'base', 'lr': 0.001, 'num_epochs': 400, 'patience_for_scheduler': 10, 'metric_to_monitor': 'Val/loss', 'metric_to_monitor_mode': 'min', 'val_check_interval': 'None', 'weight_decay': 0.0, 'use_train_augmentations': True, 'loss': 'ce', 'contrastive_temperature': 0.1, 'return_two_views': False, 'finetune_path': 'None', 'device': [0], 'max_ste